In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
path = "D:\\Github\\raw_data\\Unziper\\test.csv"
#path = "D:\\Github\\raw_data\\Unziper\\filiados_pt_sc.csv"

In [3]:
try:
    f=pd.read_csv(path, encoding ="ansi", delimiter=';',low_memory=False, skipinitialspace=True)
except:
    print("erro de leitura")  


In [4]:
keep_col = ['NOME DO FILIADO','NUMERO DA INSCRICAO','SIGLA DO PARTIDO','UF','NOME DO MUNICIPIO','ZONA ELEITORAL','SECAO ELEITORAL','DATA DA FILIACAO','SITUACAO DO REGISTRO','DATA DA DESFILIACAO']
y = f[keep_col]
       

In [5]:
y.columns = y.columns.str.replace('\D(R\$\D)','')
y.columns = y.columns.str.replace('\D(\*\D)','')
y.columns = y.columns.str.replace('\/^\s+|\s+$','')
y.columns = y.columns.str.replace(' ','_')


In [6]:
taes = y.groupby('SITUACAO_DO_REGISTRO')
taes.get_group('CANCELADO')



,NOME_DO_FILIADO,NUMERO_DA_INSCRICAO,SIGLA_DO_PARTIDO,UF,NOME_DO_MUNICIPIO,ZONA_ELEITORAL,SECAO_ELEITORAL,DATA_DA_FILIACAO,SITUACAO_DO_REGISTRO,DATA_DA_DESFILIACAO
0,ANTONIO JOSE DE OLIVEIRA,2424612640,PT,RR,BOA VISTA,1,309,21/01/2005,CANCELADO,01/10/2013
3,FRANCISCO RONNY BESSA QUEIROZ,2280962658,PT,RR,PACARAIMA,7,21,25/09/2003,CANCELADO,30/12/2011
10,ELIAS ALVES VIEIRA,3096922208,PT,RR,BOA VISTA,5,287,17/04/1996,CANCELADO,11/04/2018
14,JOAO SALUSTIANO,54852623,PT,RR,CANTÁ,3,124,08/01/2003,CANCELADO,16/04/2012


In [7]:
def situacao_registro(data_frame):
    return  data_frame.groupby('SITUACAO_DO_REGISTRO')

In [8]:
def sr_cancelado(sr_gb):
    CANCELADO = 'CANCELADO'
    df_cancelado = sr_gb.get_group(CANCELADO)
    return df_cancelado

In [9]:
def sr_regular(sr_gb):
    REGULAR = 'REGULAR'
    df_regular = sr_gb.get_group(REGULAR)
    return df_regular

In [10]:
def cancelado_grouby(df):
    not_null = df[df['DATA_DA_DESFILIACAO'].notnull()]
    is_null = df[df['DATA_DA_DESFILIACAO'].isnull()]
    return not_null,is_null


In [11]:
#main

sr_gb =  situacao_registro(y)

df_cancelado = sr_cancelado(sr_gb)

df_regular = sr_regular(sr_gb)

# quando o cancelado for null salvar esse registro em um outra tabela para  tentar processar os dados mais tardes #
cancelado_not_null,cancelado_is_null = cancelado_grouby(df_cancelado)

#cancelado_not_null
#gb_cancelado

In [16]:
cancelado_not_null

# fazer um filtro de tempo passando mes e ano 
# esse filto deve ser generico

# verificar se nos cancalados null e nos cancalados not null nao tem conflitos de id

# varificar o que fazer com os registro com mais de 100 anos



#

,NOME_DO_FILIADO,NUMERO_DA_INSCRICAO,SIGLA_DO_PARTIDO,UF,NOME_DO_MUNICIPIO,ZONA_ELEITORAL,SECAO_ELEITORAL,DATA_DA_FILIACAO,SITUACAO_DO_REGISTRO,DATA_DA_DESFILIACAO
0,ANTONIO JOSE DE OLIVEIRA,2424612640,PT,RR,BOA VISTA,1,309,21/01/2005,CANCELADO,01/10/2013
3,FRANCISCO RONNY BESSA QUEIROZ,2280962658,PT,RR,PACARAIMA,7,21,25/09/2003,CANCELADO,30/12/2011
10,ELIAS ALVES VIEIRA,3096922208,PT,RR,BOA VISTA,5,287,17/04/1996,CANCELADO,11/04/2018
14,JOAO SALUSTIANO,54852623,PT,RR,CANTÁ,3,124,08/01/2003,CANCELADO,16/04/2012


In [17]:
#df_cancelado['dat'] = np.nan
df_cancelado

#DATA_DA_DESFILIACAO


,NOME_DO_FILIADO,NUMERO_DA_INSCRICAO,SIGLA_DO_PARTIDO,UF,NOME_DO_MUNICIPIO,ZONA_ELEITORAL,SECAO_ELEITORAL,DATA_DA_FILIACAO,SITUACAO_DO_REGISTRO,DATA_DA_DESFILIACAO
0,ANTONIO JOSE DE OLIVEIRA,2424612640,PT,RR,BOA VISTA,1,309,21/01/2005,CANCELADO,01/10/2013
3,FRANCISCO RONNY BESSA QUEIROZ,2280962658,PT,RR,PACARAIMA,7,21,25/09/2003,CANCELADO,30/12/2011
10,ELIAS ALVES VIEIRA,3096922208,PT,RR,BOA VISTA,5,287,17/04/1996,CANCELADO,11/04/2018
14,JOAO SALUSTIANO,54852623,PT,RR,CANTÁ,3,124,08/01/2003,CANCELADO,16/04/2012


In [ ]:
df_re

In [ ]:
#print(type(df))
df

df.head()

In [ ]:
duplicateRowsDF = df[df.duplicated(['NOME DO FILIADO','NUMERO DA INSCRICAO','SIGLA DO PARTIDO','UF','NOME DO MUNICIPIO','ZONA ELEITORAL','SECAO ELEITORAL','DATA DA FILIACAO'])]

In [ ]:
df.loc[df['NUMERO DA INSCRICAO'] == 5957580434]
df.sort_values(by=['NUMERO DA INSCRICAO'])


In [ ]:
duplicateRowsDF = df[df.duplicated()]

In [ ]:
print(duplicateRowsDF)

In [ ]:
dr = df

In [ ]:
dr = dr.drop_duplicates(['NOME DO FILIADO','NUMERO DA INSCRICAO','SIGLA DO PARTIDO','UF','NOME DO MUNICIPIO','ZONA ELEITORAL','SECAO ELEITORAL','DATA DA FILIACAO'])

In [ ]:
dr = dr.drop_duplicates()


In [ ]:
duplicateRowsDF = dr[dr.duplicated()]
print(duplicateRowsDF)

In [ ]:
df.loc[df['NUMERO DA INSCRICAO'] == 49690240485]

In [ ]:
dr.loc[dr['SITUACAO DO REGISTRO'] != 'REGULAR']

In [ ]:
x = dr.dropna()


In [ ]:
x.count()

In [ ]:
dr.count()

In [ ]:
dr.count() - x.count()
